# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


## Install OpenPose

Build Openpose. Takes 20-30 minutes


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

#Cloning the openpose github repository and making a new project folder in drive alongwith installing Cmake to bind C++ code in Python
if 1 or not exists(project_name):
  !rm -rf openpose
  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`



install new CMake becaue of CUDA10
^C
clone openpose
^C
CMakelist.txt's caffe fix
sed: can't read openpose/CMakeLists.txt: No such file or directory
install system dependencies
build openpose
/bin/bash: line 0: cd: openpose: No such file or directory
/bin/bash: line 1:  6507 Segmentation fault      cmake -DBUILD_PYTHON=ON ..


Import Libraries

In [ ]:
#Importing all the libraries

!pip install xlrd
from os.path import exists, join, basename, splitext
import os
import cv2
from os import listdir
import shutil
from os.path import isfile, join
import sys
import cv2
import os
from sys import platform
import argparse
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import pandas as pd
import csv


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3465216/45929032 bytes (7.5%)7634944/45929032 bytes (16.6%)11862016/45929032 bytes (25.8%)16097280/45929032 bytes (35.0%)20267008/45929032 bytes (44.1%)24469504/45929032 bytes (53.3%)28704768/45929032 bytes (62.5%)32948224/45929032 bytes (71.7%)37019648/45929032 bytes (80.6%)41123840/45929032 bytes (89.5%)45285376/45929032 bytes (98.6%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

Mount Drive

In [ ]:
#This takes some times and for some reason it results in timeout for me, but still it mounts the drive even after crashing due to timeout. Proceed with running the next cell after the timeout

from google.colab import drive
drive.mount("/content/\drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g6HuyHucGYj09_NKLeTtNmmnDzPVV1D7TaC_Ykv-7OxDWeKB8R6vgw


TIMEOUT: ignored

Define Functions

In [ ]:
#This function convert videos into frames since Openpose runs on images and not videos
def get_frames(img_frame_path,video_path,video_name):
    if os.path.exists(img_frame_path):
      shutil.rmtree(img_frame_path)
    os.makedirs(img_frame_path)    
    os.chdir(video_path)
    cap = cv2.VideoCapture(video_name)
    i = 0
    p = 0
    # a variable to set how many frames you want to skip
    frame_skip = 4 #This will give a frame rate of 5 FPS, change to get different FPS
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if i > frame_skip - 1:
            os.chdir(img_frame_path)
            cv2.imwrite('test_'+str(p)+'.jpg', frame)
            i = 0
            p = p+1
            continue
        i += 1
    cap.release()
    cv2.destroyAllWindows()
    return p


#This function converts frame numbers into timestamps to be used for segmentation
def time_stamp_fn(len_frame_tot,frame_array,clip_location):

  clip = VideoFileClip(clip_location)
  duration = clip.duration
  time = duration/len_frame_tot
  print(time)
  time_stop = [time * i for i in frame_array]
  return time_stop

Main Loop (Change only the stuff in the block )

In [ ]:
######### ONLY CHANGE DIRECTORIES WITHIN THIS BLOCK #########

#Below directories change it to location where you want to keep the frames of the video. The frames will automatically delete for new videos run so be sure to clean the trash regularly for space in your drive
image_dir = r'/content/drive/My Drive/openpose_video/test1'
img_frame_path = "/content/drive/My Drive/openpose_video/test1"


#Change the directory below to the folder where you want the output
output_dir = "/content/drive/My Drive/openpose_video/output"


#Change the directory below to the folder where your videos are. Keep ONLY the files you want to process. Keep rest of the videos in a separate folder somewhere 
video_path = "/content/drive/My Drive/openpose_video/vids"



############### End of Block ###############################


#Defining an empty array to store the name of video and bounding box co-ordinates for the sign translator 
vid_name_all = []
x1_all = []
x2_all = []
y1_all = []
y2_all = []


vids = os.listdir(video_path)
print(vids)
count_video = 0
for video_name in vids:
  print(video_name)
  num_frames = get_frames(img_frame_path,video_path,video_name)
  print("Finished making frames from video")
  flag = False
  try:
      try:
          # Windows Import (Not useful since we are using Colab which is Ubuntu), always goes in else loop
          if platform == "win32":
              sys.path.append(dir_path + '/../../python/openpose/Release');
              os.environ['PATH']  = os.environ['PATH'] + ';' + dir_path + '/../../x64/Release;' +  dir_path + '/../../bin;'
              import pyopenpose as op
          else:
              sys.path.append('/content/openpose/build/python');
              from openpose import pyopenpose as op

      except ImportError as e:
          print('Error: OpenPose library could not be found. Did you enable `BUILD_PYTHON` in CMake and have this Python script in the right folder?')
          raise e

      #The below block of code is to set up Openpose, it was provided by the authors of Openpose

      # Flags
      parser = argparse.ArgumentParser()
      parser.add_argument("--image_path", default="/content/openpose/examples/media/COCO_val2014_000000000241.jpg", help="Process an image. Read all standard formats (jpg, png, bmp, etc.).")
      args = parser.parse_known_args()

      # Custom Params (refer to include/openpose/flags.hpp for more parameters)
      params = dict()
      params["model_folder"] = "/content/openpose/models"
      params["face"] = True
      params["hand"] = True

      # Add others in path?
      for i in range(0, len(args[1])):
          curr_item = args[1][i]
          if i != len(args[1])-1: next_item = args[1][i+1]
          else: next_item = "1"
          if "--" in curr_item and "--" in next_item:
              key = curr_item.replace('-','')
              if key not in params:  params[key] = "1"
          elif "--" in curr_item and "--" not in next_item:
              key = curr_item.replace('-','')
              if key not in params: params[key] = next_item

      # Starting OpenPose
      opWrapper = op.WrapperPython()
      opWrapper.configure(params)
      opWrapper.start()

      #listing down all the frames obtained from the video
      a= []
      directory = image_dir
      for filename in os.listdir(directory):
          #print(filename)
          if filename.endswith(".jpg") or filename.endswith(".png"):
              #print(os.path.join(directory, filename))
              a.append(os.path.join(directory, filename))
          else:
              continue

      #Some empty array defined so that results from each frame can be appended and worked on in the end

      pose_matrix=[]
      face_matrix = []
      lefthand_matrix = []
      righthand_matrix = []


      xminmatrix = []
      xmaxmatrix = []
      yminmatrix = []
      ymaxmatrix = []


      facearray = []
      lefthandarray = []
      righthandarray = []
      posearray = []
      frame_array = []
      frame = 0


      # Looping for current and next frmae
      for path0, path1 in zip(a,a[1:]):
        frame = frame + 1    #Increase the frame count by 1
        people_present_flag = 1 #This flag is toggled when there are some people detected in the frame, used later on to enter specific loops 

        add_distance = 0

        #Python wrapper for first image
        datum0 = op.Datum()
        imageToProcess0 = cv2.imread(path0)
        datum0.cvInputData = imageToProcess0
        opWrapper.emplaceAndPop(op.VectorDatum([datum0]))
        
        #These are the hand, pose, and face keypoints for first image
        lefthand0 = (datum0.handKeypoints[0])
        righthand0 = (datum0.handKeypoints[1])
        pose0 = datum0.poseKeypoints
        face0 = datum0.faceKeypoints 

        #Python wrapper for first image
        datum1 = op.Datum()
        imageToProcess1 = cv2.imread(path1)
        datum1.cvInputData = imageToProcess1
        opWrapper.emplaceAndPop(op.VectorDatum([datum1]))
        
        #These are the hand, pose, and face keypoints for first image
        lefthand1 = (datum1.handKeypoints[0])
        righthand1 = (datum1.handKeypoints[1])
        pose1 = datum1.poseKeypoints
        face1 = datum1.faceKeypoints

        #When no face is detected in current or next frame, there is no need to analyze hence append zeros to all arrays
        if datum0.faceKeypoints is None or datum1.faceKeypoints is None:
          facearray.append(np.zeros((70,3)))
          posearray.append(np.zeros((25,3)))
          lefthandarray.append(np.zeros((21,3)))
          righthandarray.append(np.zeros((21,3)))
          people_present_flag = 0
        

        else:
          #Otherwise find the right most keypoint of the face and see the index to who it belongs to
          rightmost = np.amax(face0[:,:,0],axis = 1)
          ###### If you expect the sign translator to be on the left side of the screen uncomment the definition of 'rightmost' below and comment out the definition of 'rightmost' above
          #rightmost = np.amin(face0[:,:,0],axis = 1)
          person_index1=np.argmax(rightmost)
          person_index = person_index1
 
          #Find keypoints X and Y of sign translator
          translatorfaceX = face0[person_index,:,0]
          translatorleftX = lefthand0[person_index,:,0]
          translatorrightX = righthand0[person_index,:,0]
          translatorposeX = pose0[person_index,:,0]
          if (face0[person_index,51,1] >0  and face0[person_index,22,1] > 0):
            add_distance = face0[person_index,51,1] - face0[person_index,22,1] 
          translatorfaceY = face0[person_index,:,1]
          translatorleftY = lefthand0[person_index,:,1]
          translatorrightY = righthand0[person_index,:,1]
          translatorposeY = pose0[person_index,:,1]


          #This loop is entered to find the bounds of sign translator (4 co-ordinates) in the image
          if (sum(translatorfaceX))>0 and (sum(translatorleftX))>0 and (sum(translatorrightX))>0 and (sum(translatorposeX))>0 and (sum(translatorfaceY))>0 and (sum(translatorleftY))>0 and (sum(translatorrightY))>0 and (sum(translatorposeY))>0 :
            #The 4 co-ordinates
            leftpointx = min(np.min(translatorfaceX[np.nonzero(translatorfaceX)]), np.min(translatorleftX[np.nonzero(translatorleftX)]),np.min(translatorrightX[np.nonzero(translatorrightX)]),np.min(translatorposeX[np.nonzero(translatorposeX)]))
            rightpointx = max(np.max(translatorfaceX[np.nonzero(translatorfaceX)]), np.max(translatorleftX[np.nonzero(translatorleftX)]),np.max(translatorrightX[np.nonzero(translatorrightX)]),np.max(translatorposeX[np.nonzero(translatorposeX)]))
            #Openpose only gives keypoints till eyebrows, so we extend it to cover the entire head
            uppointy = face0[person_index,21,1] - 0.33*(pose0[person_index,1,1] - pose0[person_index,0,1])
            downpointy = max(np.max(translatorfaceY[np.nonzero(translatorfaceY)]), np.max(translatorleftY[np.nonzero(translatorleftY)]),np.max(translatorrightY[np.nonzero(translatorrightY)]),np.max(translatorposeY[np.nonzero(translatorposeY)]))
            
            #Append all sign translator coordinates for a given frame in a matrix. We will calculate the global sign translator coordinates of a video later on 
            xminmatrix.append(leftpointx)
            xmaxmatrix.append(rightpointx)
            if (face0[person_index,21,1]>0) and (pose0[person_index,1,1])>0 and (pose0[person_index,0,1])>0:
              yminmatrix.append(uppointy)
            ymaxmatrix.append(downpointy)

          #Append the keypoints of translator in array. Later we will segment and use them for embeddings
          facearray.append(face0[person_index,:,:])
          posearray.append(pose0[person_index,:,:])
          lefthandarray.append(lefthand0[person_index,:,:])
          righthandarray.append(righthand0[person_index,:,:])

        
        #This loop is for the segmentation, based on movement of sign translator in current and next frame
        if people_present_flag == 1:
          rightmost0 = np.amax(face0[:,:,0],axis = 1)
          ###### If you expect the sign translator to be on the left side of the screen uncomment the definition of 'rightmost' below and comment out the definition of 'rightmost' above
          #rightmost0 = np.amin(face0[:,:,0],axis = 1)
          person_index0=np.argmax(rightmost0)
          rightmost1 = np.amax(face1[:,:,0],axis = 1)
          ###### If you expect the sign translator to be on the left side of the screen uncomment the definition of 'rightmost' below and comment out the definition of 'rightmost' above          
          #rightmost1 = np.amin(face1[:,:,0],axis = 1)
          person_index1=np.argmax(rightmost1)

          #Distance between head and chest in pose keypoints. Useful to remove cases where the sign translator doesnt move much between frames but is not a stop frame
          reference_distancey = pose0[person_index0,1,1] - pose0[person_index0,0,1] 

          #Mean of visible keypoints of the hands of the sign translator
          lefthandxmean0 = np.sum(lefthand0[person_index0,:,0])/(np.count_nonzero(lefthand0[person_index0,:,0]))
          lefthandymean0 = np.sum(lefthand0[person_index0,:,1])/(np.count_nonzero(lefthand0[person_index0,:,1]))
          righthandxmean0 = np.sum(righthand0[person_index0,:,0])/(np.count_nonzero(righthand0[person_index0,:,0]))
          righthandymean0 = np.sum(righthand0[person_index0,:,1])/(np.count_nonzero(righthand0[person_index0,:,1]))


          lefthandxmean1 = np.sum(lefthand1[person_index1,:,0])/(np.count_nonzero(lefthand1[person_index1,:,0]))
          lefthandymean1 = np.sum(lefthand1[person_index1,:,1])/(np.count_nonzero(lefthand1[person_index1,:,1]))
          righthandxmean1 = np.sum(righthand1[person_index1,:,0])/(np.count_nonzero(righthand1[person_index1,:,0]))
          righthandymean1 = np.sum(righthand1[person_index1,:,1])/(np.count_nonzero(righthand1[person_index1,:,1]))


          #Distance between hands of the sign translator in frame
          hand_distancex = abs(lefthandxmean0 - righthandxmean0) 
          hand_distancey = abs(lefthandymean0 - righthandymean0) 
          lefthand_frompose = abs(lefthandymean0 - pose0[person_index0,1,1])
          righthand_frompose =  abs(righthandymean0 - pose0[person_index0,1,1])

          #Change in the position of hands between consecutive frames
          lefthandmeanx_change = abs((lefthandxmean1-lefthandxmean0)/lefthandxmean0)
          lefthandmeany_change = abs((lefthandymean1-lefthandymean0)/lefthandymean0)
          righthandmeanx_change = abs((righthandxmean1-righthandxmean0)/righthandxmean0)
          righthandmeany_change = abs((righthandymean1-righthandymean0)/righthandymean0)



          tolerance = 0.02 #How much percent change is tolerated to be in the current state (Segment/Not Segment)?

          #Loop that determines when a frame is a start frame
          if ((lefthandmeanx_change)> tolerance or (lefthandmeany_change) > tolerance or (righthandmeanx_change) > tolerance or (righthandmeany_change) > tolerance) and flag == False:
            flag = True
            frame_array.append(frame)

          #Loop that determines when a frame is a stop frames. Alongwith minimal change in the coordinates, the distance based rules defined earlier have to be satisfied to avoid false positives
          if (lefthandmeanx_change)< tolerance and (lefthandmeany_change) < tolerance and (righthandmeanx_change) < tolerance and (righthandmeany_change) < tolerance and flag == True and hand_distancex < 0.15*(reference_distancey) and hand_distancey < 0.15*(reference_distancey) and lefthand_frompose > 0.75*(reference_distancey) and righthand_frompose > 0.75*(reference_distancey):
            frame_array.append(frame)
            flag = False
          

      #Calculate the global coordinates for the sign language translator in a video
      x1=min(xminmatrix)
      y1=min(yminmatrix)
      if y1<0:
        y1 = 0
      x2=max(xmaxmatrix)
      y2=max(ymaxmatrix)

      #Append the information for all the videos in the loop
      vid_name_all.append(video_name)
      x1_all.append(x1)
      x2_all.append(x2)
      y1_all.append(y1)
      y2_all.append(y2)

      face_tensor = np.stack(facearray)
      pose_tensor = np.array(posearray)
      lefthand_tensor = np.array(lefthandarray)
      righthand_tensor = np.array(righthandarray)


      #Save information about the keypoints of the sign translator in a video in 4 numpy arrays
      np.save(output_dir+"/"+(video_name.split('.')[0])+"_face_tensor",face_tensor)
      np.save(output_dir+"/"+(video_name.split('.')[0])+"_pose_tensor",pose_tensor)
      np.save(output_dir+"/"+(video_name.split('.')[0])+"_lefthand_tensor",lefthand_tensor)
      np.save(output_dir+"/"+(video_name.split('.')[0])+"_righthand_tensor",righthand_tensor)

      #Divide start/stop frame numbers into timestamps
      len_frame_tot = len(frame_array)
      final_time = []
      clip_location = video_path+"/"+video_name
      time_stop = time_stamp_fn(num_frames,frame_array,clip_location)

      start_times = time_stop[0:][::2]
      end_times = time_stop[1:][::2]
      if len(start_times) != len(end_times):
        end_times.append('NaN')

      timestamp_final = list(zip(start_times,end_times))  
      timestamp_final = np.asarray(timestamp_final,dtype='float64')
      timestamp_final_filtered = timestamp_final[ (timestamp_final[:,1] - timestamp_final[:,0]) > 2] #Remove segments less than 2 seconds
      print(timestamp_final_filtered) 
      np.save(output_dir+"/"+(video_name.split('.')[0])+"_timestamp", timestamp_final_filtered) #Save final timestamps in numpy array
      count_video = count_video + 1
      print("Finished processing Video ", count_video)

  except Exception as e:
      print(e)
      sys.exit(-1)

#Save the sign translator coordinates of all videos in a csv file
sign_translator_list = list(zip(vid_name_all,x1_all,x2_all,y1_all,y2_all))
sign_translator_df = pd.DataFrame(sign_translator_list)
sign_translator_df.columns =['Video_Name', 'x1', 'x2', 'y1','y2']
sign_translator_df.to_csv(output_dir+"/sign_translator_location.csv",index=None)
print("Finished processing All videos")


['video38.ts', 'video44.ts', 'video43.ts', 'video41.ts']
video38.ts
Finished making frames from video
/content/drive/My Drive/frames/test_0.jpg
/content/drive/My Drive/frames/test_1.jpg
/content/drive/My Drive/frames/test_1.jpg
/content/drive/My Drive/frames/test_2.jpg
/content/drive/My Drive/frames/test_2.jpg
/content/drive/My Drive/frames/test_3.jpg
/content/drive/My Drive/frames/test_3.jpg
/content/drive/My Drive/frames/test_4.jpg
/content/drive/My Drive/frames/test_4.jpg
/content/drive/My Drive/frames/test_5.jpg


/content/drive/My Drive/frames/test_5.jpg
/content/drive/My Drive/frames/test_6.jpg
/content/drive/My Drive/frames/test_6.jpg
/content/drive/My Drive/frames/test_7.jpg
/content/drive/My Drive/frames/test_7.jpg
/content/drive/My Drive/frames/test_8.jpg
/content/drive/My Drive/frames/test_8.jpg
/content/drive/My Drive/frames/test_9.jpg
/content/drive/My Drive/frames/test_9.jpg
/content/drive/My Drive/frames/test_10.jpg
/content/drive/My Drive/frames/test_10.jpg
/content/drive/My Drive/frames/test_11.jpg
/content/drive/My Drive/frames/test_11.jpg
/content/drive/My Drive/frames/test_12.jpg
/content/drive/My Drive/frames/test_12.jpg
/content/drive/My Drive/frames/test_13.jpg
/content/drive/My Drive/frames/test_13.jpg
/content/drive/My Drive/frames/test_14.jpg
/content/drive/My Drive/frames/test_14.jpg
/content/drive/My Drive/frames/test_15.jpg
/content/drive/My Drive/frames/test_15.jpg
/content/drive/My Drive/frames/test_16.jpg
/content/drive/My Drive/frames/test_16.jpg
/content/drive/My Dr

KeyboardInterrupt: ignored